<a href="https://colab.research.google.com/github/sidhu2690/ai-from-scratch/blob/main/01_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np

class Conv2D:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / (filter_size**2)

    def forward(self, input):
        self.last_input = input
        h, w = input.shape
        f = self.filter_size

        output = np.zeros((h - f + 1, w - f + 1, self.num_filters))

        for i in range(h - f + 1):
            for j in range(w - f + 1):
                region = input[i:i+f, j:j+f]
                for k in range(self.num_filters):
                    output[i, j, k] = np.sum(region * self.filters[k])

        return output

    def backward(self, d_out, lr):
        """
        d_out shape = (H-f+1, W-f+1, num_filters)
        """
        h, w = self.last_input.shape
        f = self.filter_size

        d_filters = np.zeros_like(self.filters)
        d_input = np.zeros_like(self.last_input)

        for i in range(h - f + 1):
            for j in range(w - f + 1):
                region = self.last_input[i:i+f, j:j+f]

                for k in range(self.num_filters):
                    # gradient w.r.t filters
                    d_filters[k] += d_out[i, j, k] * region

                    # gradient w.r.t input
                    d_input[i:i+f, j:j+f] += d_out[i, j, k] * self.filters[k]

        # update filters
        self.filters -= lr * d_filters

        return d_input


In [39]:
conv = Conv2D(num_filters=2, filter_size=3)

image = np.random.randn(5,5)

out = conv.forward(image)

d_out = np.random.randn(*out.shape)

d_input = conv.backward(d_out, lr=0.01)

print(out.shape)
print(d_input.shape)


(3, 3, 2)
(5, 5)


In [40]:
class ReLU:
    def forward(self, x):
        self.x = x
        return np.maximum(0, x)

    def backward(self, d_out):
        return d_out * (self.x > 0)


In [41]:
class Dense:
    def __init__(self, in_dim, out_dim):
        self.W = np.random.randn(out_dim, in_dim) * 0.01
        self.b = np.zeros((out_dim,1))

    def forward(self, x):
        self.x = x
        return self.W @ x + self.b

    def backward(self, d_out, lr):
        dW = d_out @ self.x.T
        db = d_out
        dx = self.W.T @ d_out

        self.W -= lr * dW
        self.b -= lr * db

        return dx


In [47]:
def softmax(x):
    e = np.exp(x - np.max(x))
    return e / np.sum(e)

def cross_entropy(pred, label):
    return float(-np.log(pred[label] + 1e-9))



In [48]:
conv = Conv2D(8, 3)     # output → (26,26,8)
relu = ReLU()
dense = Dense(26*26*8, 10)


In [49]:
def forward(image):
    out = conv.forward(image)          # (26,26,8)
    out = relu.forward(out)
    out = out.reshape(-1,1)            # flatten
    out = dense.forward(out)
    out = softmax(out)
    return out
def backward(pred, label, lr):
    d_out = pred.copy()
    d_out[label] -= 1                   # softmax derivative

    d_out = dense.backward(d_out, lr)   # (26*26*8, 1)
    d_out = d_out.reshape(26,26,8)
    d_out = relu.backward(d_out)
    conv.backward(d_out, lr)


In [50]:
from keras.datasets import mnist

(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0


In [51]:
lr = 0.01

for epoch in range(2):
    loss = 0
    correct = 0

    for i in range(500):
        x = trainX[i]
        y = trainY[i]

        pred = forward(x)
        loss += cross_entropy(pred, y)

        if np.argmax(pred) == y:
            correct += 1

        backward(pred, y, lr)

    print(f"Epoch {epoch+1}, Loss: {loss/500:.4f}, Acc: {correct/500:.4f}")


/tmp/ipython-input-2868959615.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(-np.log(pred[label] + 1e-9))


Epoch 1, Loss: 1.0568, Acc: 0.6500
Epoch 2, Loss: 0.4613, Acc: 0.8420
